In [1]:
import pandas as pd
import numpy as np
import re
from string import punctuation
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("./data_twitter/tweets_train.csv",encoding='utf-8')
train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [3]:
print train.shape
print validate.shape
print test.shape

(293159, 4)
(97720, 4)
(97720, 4)


In [4]:
embeddings={}
with open('./glove.twitter.27B/glove.twitter.27B.200d.txt','r') as f:
    for line in f:
        list_line = line.split()
        word= list_line[0]
        word= word.replace('<',' ')
        word = word.replace('>', ' ')
        embeddings[word] = np.array(list_line[1:],dtype=np.float16)

In [5]:
def _tweets_preprocess(tweets):
    tweets_clean=np.zeros_like(tweets)
    for i,tweet in enumerate(tweets):
        p = re.compile(r'(\n)|(\r)|(\t)|(\')|(\u00A9)|([!"#$%&()*+,-./:;<=>?@\[\\\]^_`{|}~])', re.IGNORECASE)
        tweet = re.sub('\?', ' ', tweet)
        tweet = re.sub('\.', ' ', tweet)
        tweet = re.sub(',', ' ', tweet)
        tweet = re.sub('!', ' ', tweet)
        tweet = re.sub(' +',' ', tweet)
        tweet = re.sub(p," ",tweet)
        tweet= tweet.encode('ascii','ignore')

        tweet_clean = [ wd.strip(punctuation).lower() for wd in tweet.split() \
                    if not wd.startswith('@') and not wd.startswith('#') and not wd == 'rt']
        #wd[0].isupper()
        
        tweets_clean[i]= np.array(tweet_clean)
    return tweets_clean

In [6]:
X_train= _tweets_preprocess(train['text'].values)
y_train= train['label'].values
X_val= _tweets_preprocess(validate['text'].values)
y_val= validate['label'].values
X_test= _tweets_preprocess(test['text'].values)
y_test= test['label'].values

In [7]:
def _calc_embeddings(X):
    embeddings_data= np.zeros((X.shape[0],200))
    for i,tweet in enumerate(X):
        words=0
        for token in tweet:
            words+=1.0
            try:
                embeddings_data[i]+=embeddings[token.lower().strip()]
            except:
                continue
        embeddings_data[i]/=words
    return embeddings_data

In [8]:
embeddings_train= _calc_embeddings(X_train)
embeddings_val= _calc_embeddings(X_val)
embeddings_test= _calc_embeddings(X_test)

In [9]:
encoder = MultiLabelBinarizer()
encoder.fit(y_train[:,np.newaxis])
y_train = encoder.transform(y_train[:,np.newaxis])
y_val = encoder.transform(y_val[:,np.newaxis])
y_test  = encoder.transform(y_test[:,np.newaxis])

In [10]:
print embeddings_train.shape
print y_train.shape

(293159, 200)
(293159, 20)


In [25]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K
from keras.callbacks import History 

In [26]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


In [27]:
def createmodel():
    model = Sequential()
    model.add(Dense(512,input_shape=(200,),activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(32,activation='relu'))
    #model.add(Dense(32,activation='relu'))
    model.add(Dense(20,activation='softmax'))
    model.summary()
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[f1,'accuracy'])
    return model


In [28]:
model=createmodel()
history = History()
res = model.fit(embeddings_train, y_train, batch_size=512, epochs=70, validation_data=(embeddings_val, y_val),
                callbacks=[history])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               102912    
_________________________________________________________________
dense_20 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_21 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 20)                660       
Total params: 278,132
Trainable params: 278,132
Non-trainable params: 0
_________________________________________________________________
Trai

In [30]:
model.save('nn_model.h5')

In [31]:
from keras.models import load_model

In [32]:
modl= load_model('nn_model.h5',custom_objects={'f1': f1})

In [37]:
history=res

In [39]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['f1'])
plt.plot(history.history['val_f1'])
plt.title('model f1 score')
plt.ylabel('f1')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

['acc', 'f1', 'loss', 'val_acc', 'val_f1', 'val_loss']
